# Capstone Project

**IBDM Dataset**
This capstone is divided into a 4-phase duration. In this document, the entire details of the 4-phase(weekly) duration are given.
**Phase 1**

**1) Webscrape the provided URL - IMDB dataset:** https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1

**2) Store the entire data in two different CSV files as per the given fields:**
- The first CSV file data contains : Sno, Movie Name, Director Name, Duration, year, ratings, Metascore Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2
- The second CSV file contains the following: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4

Bifurcate the genre into 3 subfields as per the number of genres the movie belongs to such as : Genre1, genre2, genre3 Phase 2

Make two tables and corresponding columns provided in the above CSV files in SQLite DBMS. Insert all data of each CSV file in each of the created tables. Now start querying the table(s) in the SQL workbench / SQLite database :

**Table 1: Sno, MovieName, Director Name, Duration, genre, ratings**

- Display all the details of movies created by directors Christopher and Matt Reeves.
- Display all the details of movies with a duration of 140 minutes to 190 minutes.
- Display all details of movies with ratings above 7 in ascending order.
- Display all movie names in descending order.
- Display movie name starts with ‘P’ and their rating is greater than 7.

<b>Table 2: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification<b>

- Display all movie names with star Arnold Schwarzenegger in ascending order.
- Display all details of the movie with the highest number of votes.
- Display movie names with gross collections in descending order.
- Display the gross collection of movies with the star Arnold.
- Display all details of movies with comedy and action genres.

<b>Make subquery :<b>

- Display all details from both tables where movie names are the same.
- Display all movie names, Director, ratings, and gross collection where the genre is action.
- Display all details from both tables with the highest gross collection.
- Display all details from both tables with the highest ratings
- Display all details from both tables with the lowest gross collection and lowest ratings

Now once completed with queries in the SQLite database, then make the exact query solutions by using PANDAS SQL in the data frame. Load the CSV data in a data frame and start making solutions for all the above 15 queries using PANDAS SQL. You may use concat or merge joins per the requirements basis to make 5 join queries.

**Phase 3**
    
Now make only 1 data frame of two CSV files using the join operation of pandas and start doing EDA.
Do the complete EDA in detail to explore the insights of data and write detailed observations of each analysis.

**Phase 4**
    
- Write the complete Machine learning code to make **predictions of votes and gross collection**. Use appropriate models on their label basis. Remember you need to make **2 different predictions: vote and gross collection.**
- Apply all the best techniques of scaling, and hyperparameter tuning, and avoid underfitting or overfitting (bias/variance)
- At the end save the best model and convey on which basis you have chosen that model.

In [160]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import csv
import sqlite3

In [161]:

url = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'
response = requests.get(url)
print(response)

<Response [200]>


- The first CSV file data contains : Sno, Movie Name, Director Name, Duration, year, ratings, Metascore Bifurcate the Director field into subfields as per the number of directors of the movie belongs to such as Director1, director2

In [3]:
# scrapping code to collect the data in csv file

Sno=[]
MoviesName=[]
Director_1=[]
Director_2=[]
Director_3=[]
Duration=[]
Metascore=[]
Year=[]
Ratings=[]
   
pages = np.arange(1,1750,50)


for page in  tqdm(pages):
    
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=" + str(page) + "&ref_=adv_nxt")
  
    soup=BeautifulSoup(page.text, 'html.parser')
  
    movies=soup.find('div',class_="lister-list").find_all('div',class_='lister-item-content')
  
    sleep(2)
    
    for movie in movies:
        sno=movie.find('h3' ,class_="lister-item-header").span.text.strip('.').replace(',','')
        Sno.append(int(sno))
        movie_name=movie.find('h3' ,class_="lister-item-header").a.text
        MoviesName.append(movie_name)
        director=movie.find('p',class_="").get_text(strip=True).split('|')[0]
        if len(director.split(','))==3:
            director_1=director.split(',')[0].strip(' Directors: ')
            director_2=director.split(',')[1]
            director_3=director.split(',')[2]
            Director_1.append(director_1)
            Director_2.append(director_2)
            Director_3.append(director_3)
        
        elif len(director.split(','))==2:
            director_1=director.split(',')[0].strip(' Directors: ')
            director_2=director.split(',')[1]
            Director_1.append(director_1)
            Director_2.append(director_2)
            Director_3.append(np.nan)
            
            
        else:
            director_1=director.strip(' Directors: ')
            Director_1.append(director_1)
            Director_2.append(np.nan)
            Director_3.append(np.nan)
            
        
        duration=movie.find('span',class_="runtime").text.strip(' min')
        Duration.append(duration)
        year=movie.find('span',class_="lister-item-year text-muted unbold").text.strip('(I) ').strip('()')
        Year.append(year)
        ratings=movie.find('div', class_="inline-block ratings-imdb-rating").strong.text.strip('.')
        Ratings.append(ratings)
        try:
            metascore=movie.find('div', class_="inline-block ratings-metascore").span.get_text(strip=True)
            Metascore.append(metascore)
        except Exception as e:
            metascore=np.nan
            Metascore.append(metascore)
        
print(Sno,MoviesName,Director_1,Director_2,Director_3,Duration,Year,Ratings,Metascore)

100%|███████████████████████████████████████████| 35/35 [02:19<00:00,  3.97s/it]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [162]:
print(len(Sno))

1750


In [163]:
Director_1

['Christopher Nolan',
 'Peter Jackson',
 'Joaquim Dos San',
 'Christopher Nolan',
 'Peter Jackson',
 'Peter Jackson',
 'Lana Wachowsk',
 'Irvin Kershn',
 'Sudha Kongara',
 'James Cameron',
 'George Luca',
 'Akira Kurosawa',
 'Masaki Kobayash',
 'Hanu Raghavapud',
 'Ridley S',
 'Luc Besson',
 'Steven Spielberg',
 'Bob Persich',
 'Christopher Nolan',
 'Anthony Ru',
 'James Cameron',
 'Park Chan-wook',
 'Anthony Ru',
 'Lokesh Kanagaraj',
 'Vetrimaaran',
 'Joseph Kosinsk',
 'Michael Mann',
 'Richard Marquand',
 'Hayao Miyazak',
 'Alfred Hitchcock',
 'Nitesh Tiwa',
 'Rishab Shetty',
 'Prashanth Neel',
 'Lokesh Kanagaraj',
 'Vishnuvardhan',
 'Ram Kuma',
 'Steven Spielberg',
 'Steven Spielberg',
 'Christopher Nolan',
 'Jon Wa',
 'Sam Mend',
 'John McTiernan',
 'Quentin Tarantin',
 'James McTeigu',
 'Akira Kurosawa',
 'Haruo Sotozak',
 'S.S. Rajamoul',
 'Anurag Kashyap',
 'Akira Kurosawa',
 'Aditya Dha',
 'Sukuma',
 'Prashanth Neel',
 'Buster Keaton',
 'Alper Cagla',
 'Gaya',
 'Tigmanshu Dhuli

In [164]:
df= pd.DataFrame({'Name':MoviesName,'Director_1':Director_1,'Director_2':Director_2,'Director_3':Director_3,'Duration':Duration,'Metascore':Metascore,'Year':Year,'Ratings':Ratings})

In [165]:
df

,Name,Director_1,Director_2,Director_3,Duration,Metascore,Year,Ratings
0,The Dark Knight,Christopher Nolan,NaN,NaN,152,84,2008,9.0
1,The Lord of the Rings: The Return of the King,Peter Jackson,NaN,NaN,201,94,2003,9.0
2,Spider-Man: Across the Spider-Verse,Joaquim Dos San,Kemp Powers,Justin K. Thompson,140,86,2023,8.9
3,Inception,Christopher Nolan,NaN,NaN,148,74,2010,8.8
4,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,NaN,NaN,178,92,2001,8.8
...,...,...,...,...,...,...,...,...
1745,The Adventures of Sharkboy and Lavagirl 3-D,Robert Rodriguez,NaN,NaN,93,38,2005,3.7
1746,Superman IV: The Quest for Peace,Sidney J. Fu,NaN,NaN,90,24,1987,3.7
1747,Dungeons & Dragons,Courtney Solomon,NaN,NaN,107,14,2000,3.6
1748,Mortal Kombat: Annihilation,John R. Leon,NaN,NaN,95,11,1997,3.6


In [166]:
df.to_csv('Movies_1.csv')

### 2nd CSV

- The second CSV file contains the following: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4


In [9]:
MoviesName = []
Stars_1 = []
Stars_2 = []
Stars_3 = []
Stars_4 = []
Votes = []
Gross = []
Genre_1 = []
Genre_2 = []
Genre_3 = []
Certification = []
Popularity = []

pages = np.arange(1, 1750, 50)  # Contain Values 1 tp 1749 , with step of 50

for page in  tqdm(pages):     #  fast and extensible progress bar for loops and other iterable processes
    url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start={}&ref_=adv_nxt".format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movies = soup.find('div', class_="lister-list").find_all('div', class_='lister-item-content')
    #sleep(1)

    for movie in movies:
        movie_name = movie.find('h3', class_="lister-item-header").a.text
        MoviesName.append(movie_name)

        stars = movie.find('p', class_="").get_text(strip=True).split('|')[1].split(',')
        Stars_1.append(stars[0].strip(' Stars: '))
        Stars_2.append(stars[1].strip() if len(stars) >= 2 else np.nan)
        Stars_3.append(stars[2].strip() if len(stars) >= 3 else np.nan)
        Stars_4.append(stars[3].strip() if len(stars) >= 4 else np.nan)

        vote = movie.find('p', class_="sort-num_votes-visible").get_text(strip=True).split('|')[0][6:]
        Votes.append(vote)

        genre = movie.find('span', class_="genre").get_text(strip=True).split(',')
        
        Genre_1.append(genre[0].strip() if len(genre) >= 1 else np.nan)
        Genre_2.append(genre[1].strip() if len(genre) >= 2 else np.nan)
        Genre_3.append(genre[2].strip() if len(genre) >= 3 else np.nan)

        try:
            gross = movie.find('p', class_="sort-num_votes-visible").get_text(strip=True).split('|')[1][6:].replace('$', '').replace('M', '')
        except IndexError:
            gross = np.nan
        Gross.append(gross)

        try:
            certificate = movie.find('span', class_="certificate").text
        except AttributeError:
            certificate = np.nan
        Certification.append(certificate)
    
        href=movie.find('h3' ,class_="lister-item-header").a.get('href')
        popularity_url=  f"https://www.imdb.com{href}"
        # Connecting to the webdriver
        # Create ChromeDriver instance using WebDriverManager
        driver = webdriver.Chrome()
        # Opening www.imdb.comin in chrome browser
        driver.get(popularity_url)
        try:
            pop=driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]')
            Popularity.append(pop.text)
            
        except NoSuchElementException:
            Popularity.append(np.nan)  
        

print(MoviesName,Stars_1,Stars_2,Stars_3,Stars_4,Votes,Gross,Genre_1,Genre_2,Genre_3,Certification,Popularity)
# Close the WebDriver
driver.close()

100%|████████████████████████████████████████| 35/35 [8:30:10<00:00, 874.59s/it]


['The Dark Knight', 'The Lord of the Rings: The Return of the King', 'Spider-Man: Across the Spider-Verse', 'Inception', 'The Lord of the Rings: The Fellowship of the Ring', 'The Lord of the Rings: The Two Towers', 'The Matrix', 'Star Wars: Episode V - The Empire Strikes Back', 'Soorarai Pottru', 'Terminator 2: Judgment Day', 'Star Wars', 'Shichinin no samurai', 'Seppuku', 'Sita Ramam', 'Gladiator', 'Léon', 'Raiders of the Lost Ark', 'Spider-Man: Into the Spider-Verse', 'The Dark Knight Rises', 'Avengers: Endgame', 'Aliens', 'Oldeuboi', 'Avengers: Infinity War', 'Kaithi', 'Asuran', 'Top Gun: Maverick', 'Heat', 'Star Wars: Episode VI - Return of the Jedi', 'Mononoke-hime', 'North by Northwest', 'Dangal', 'Kantara', 'K.G.F: Chapter 2', 'Vikram', 'Shershaah', 'Ratsasan', 'Indiana Jones and the Last Crusade', 'Jurassic Park', 'Batman Begins', 'Spider-Man: No Way Home', '1917', 'Die Hard', 'Kill Bill: Vol. 1', 'V for Vendetta', 'Ran', 'Kimetsu no Yaiba: Mugen Ressha-Hen', 'Baahubali 2: The 

In [167]:
print(len(MoviesName))

1750


In [168]:
df1= pd.DataFrame({'Popular':Popularity,'Name':MoviesName,'Stars_1': Stars_1, 'Stars_2':Stars_2, 'Stars_3':Stars_3, 'Stars_4':Stars_4, 'Votes': Votes, 'Genre_1':Genre_1, 'Genre_2':Genre_2, 'Genre_3':Genre_3, 'Gross':Gross, 'Certification':Certification})

In [169]:
df1

,Popular,Name,Stars_1,Stars_2,Stars_3,Stars_4,Votes,Genre_1,Genre_2,Genre_3,Gross,Certification
0,39,The Dark Knight,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"2,750,814",Action,Crime,Drama,534.86,UA
1,311,The Lord of the Rings: The Return of the King,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,"1,902,836",Action,Adventure,Drama,377.85,U
2,13,Spider-Man: Across the Spider-Verse,hameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,"195,199",Animation,Action,Adventure,NaN,U
3,43,Inception,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,"2,441,597",Action,Adventure,Sci-Fi,292.58,UA
4,152,The Lord of the Rings: The Fellowship of the Ring,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,"1,931,115",Action,Adventure,Drama,315.54,U
...,...,...,...,...,...,...,...,...,...,...,...,...
1745,"2,717",The Adventures of Sharkboy and Lavagirl 3-D,Cayden Boyd,George Lopez,Kristin Davis,David Arquette,"40,390",Action,Adventure,Comedy,39.18,U
1746,"4,213",Superman IV: The Quest for Peace,Christopher Reeve,Gene Hackman,Margot Kidder,Jackie Cooper,"51,155",Action,Adventure,Sci-Fi,15.68,U
1747,"3,175",Dungeons & Dragons,Justin Whalin,Jeremy Irons,Zoe McLellan,Bruce Payne,"35,414",Action,Adventure,Fantasy,15.22,UA
1748,"4,875",Mortal Kombat: Annihilation,Robin Shou,Talisa Soto,James Remar,Sandra Hess,"56,771",Action,Adventure,Fantasy,35.93,UA


In [170]:
df1.to_csv('Movies_2.csv',index=False, header=False)

In [171]:
# create a databse with name  Movies.db
conn= sqlite3.connect('Movies.db')

# cursor object
cursor = conn.cursor()

In [172]:
# create Table name = movie_1

table1= """ CREATE TABLE movie_1 (
            Serial_no INTEGER NOT NULL,
            Movie_Name VARCHAR(255) ,
            Director_1 VARCHAR(255),
            Director_2 VARCHAR(255),
            Director_3 VARCHAR(255),
            Duration INTEGER,
            Year TEXT,
            Ratings FLOAT,
            Metascore FLOAT,
            PRIMARY KEY (Serial_no,Movie_Name)
        ); """

cursor.execute(table1)
 
print("Table1 is Ready")

Table1 is Ready


- The second CSV file contains the following: Movie Name, stars, votes, Genre, Gross collection, popularity, Certification Bifurcate the stars field into 4 subfields as per the number of stars worked in the movie such as star1, star2, star3, star4


In [173]:
# create Table name = movie_2

table2= """ CREATE TABLE movie_2 (
            MoviesName VARCHAR(250),
            Stars_1 VARCHAR(255),
            Stars_2 VARCHAR(255),
            Stars_3 VARCHAR(255),
            Stars_4 VARCHAR(255),
            Votes INTEGER,
            Genre_1 VARCHAR(25),
            Genre_2 VARCHAR(25),
            Genre_3 VARCHAR(25),
            Gross FLOAT,
            Certification VARCHAR(25),
            Popularity INTEGER,
            FOREIGN KEY (MoviesName) REFERENCES movie_1 (Movie_Name)
        ); """

cursor.execute(table2)
 
print("Table2 is Ready")

Table2 is Ready


In [174]:
# Opening the movie_1.csv file
file = open('Movies_1.csv',encoding='utf-8')
 
# Reading the contents of the

contents = csv.reader(file)
next(contents)
# SQL query to insert data into the table
insert_records = " INSERT INTO movie_1 (Serial_no, Movie_Name, Director_1,Director_2,Director_3, Duration, Year, Ratings, Metascore) VALUES(?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
conn.commit()

# SQL query to retrieve all data from the table
select_all = "SELECT * FROM movie_1"
rows = cursor.execute(select_all).fetchall()

cursor.lastrowid

1750

In [175]:
# Opening the movie_2.csv file
file = open('Movies_2.csv',encoding='utf-8')
 
# Reading the contents of the

contents = csv.reader(file)
next(contents)
# SQL query to insert data into the table
insert_records = " INSERT INTO movie_2 (MoviesName, Stars_1,Stars_2,Stars_3,Stars_4,Votes,Genre_1,Genre_2,Genre_3,Gross,Certification,Popularity) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file into our table
cursor.executemany(insert_records, contents)
conn.commit()

# SQL query to retrieve all data from the table
select_all = "SELECT * FROM movie_2"
rows = cursor.execute(select_all).fetchall()
conn.close()

cursor.lastrowid

1749

In [176]:
# 1) Display all the details of movies created by directors Christopher and Matt Reeves. 
conn= sqlite3.connect('Movies.db')

# cursor object
mycursor = conn.cursor()
mycursor.execute("SELECT * FROM movie_1 WHERE Director_1  in ('Christopher Nolan','Matt Reev');")
myresult=mycursor.fetchall()

for x in myresult:
    print(x)

(0, 'The Dark Knight', 'Christopher Nolan', '', '', 152, '84', 2008.0, 9.0)
(3, 'Inception', 'Christopher Nolan', '', '', 148, '74', 2010.0, 8.8)
(18, 'The Dark Knight Rises', 'Christopher Nolan', '', '', 164, '78', 2012.0, 8.4)
(38, 'Batman Begins', 'Christopher Nolan', '', '', 140, '70', 2005.0, 8.2)
(130, 'Dunkirk', 'Christopher Nolan', '', '', 106, '94', 2017.0, 7.8)
(131, 'The Batman', 'Matt Reev', '', '', 176, '72', 2022.0, 7.8)
(227, 'Dawn of the Planet of the Apes', 'Matt Reev', '', '', 130, '79', 2014.0, 7.6)
(312, 'War for the Planet of the Apes', 'Matt Reev', '', '', 140, '82', 2017.0, 7.4)
(324, 'Tenet', 'Christopher Nolan', '', '', 150, '69', 2020.0, 7.3)
(516, 'Cloverfield', 'Matt Reev', '', '', 85, '64', 2008.0, 7.0)


In [177]:
# 2) Display all the details of movies with a duration of 140 minutes to 190 minutes.
mycursor.execute("SELECT * FROM movie_1 WHERE Duration BETWEEN 140 and 190;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(0, 'The Dark Knight', 'Christopher Nolan', '', '', 152, '84', 2008.0, 9.0)
(2, 'Spider-Man: Across the Spider-Verse', 'Joaquim Dos San', 'Kemp Powers', 'Justin K. Thompson', 140, '86', 2023.0, 8.9)
(3, 'Inception', 'Christopher Nolan', '', '', 148, '74', 2010.0, 8.8)
(4, 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', '', '', 178, '92', 2001.0, 8.8)
(5, 'The Lord of the Rings: The Two Towers', 'Peter Jackson', '', '', 179, '87', 2002.0, 8.8)
(8, 'Soorarai Pottru', 'Sudha Kongara', '', '', 153, '', 2020.0, 8.7)
(13, 'Sita Ramam', 'Hanu Raghavapud', '', '', 163, '', 2022.0, 8.6)
(14, 'Gladiator', 'Ridley S', '', '', 155, '67', 2000.0, 8.5)
(18, 'The Dark Knight Rises', 'Christopher Nolan', '', '', 164, '78', 2012.0, 8.4)
(19, 'Avengers: Endgame', 'Anthony Ru', 'Joe Russo', '', 181, '78', 2019.0, 8.4)
(22, 'Avengers: Infinity War', 'Anthony Ru', 'Joe Russo', '', 149, '68', 2018.0, 8.4)
(23, 'Kaithi', 'Lokesh Kanagaraj', '', '', 145, '', 2019.0, 8.4)
(24, 'Asuran', '

In [178]:
# 3) Display all details of movies with ratings above 7 in ascending order.
mycursor.execute("SELECT * FROM movie_1 where Ratings >7.0 ORDER by Ratings;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(52, 'Sherlock Jr.', 'Buster Keaton', '', '', 45, '', 1924.0, 8.2)
(72, 'The General', 'Clyde Bruckman', 'Buster Keaton', '', 78, '', 1926.0, 8.1)
(195, 'Scarface', 'Howard Hawk', 'Richard Rosson', '', 93, '90', 1932.0, 7.7)
(125, 'The Adventures of Robin Hood', 'Michael Curtiz', 'William Keighley', '', 102, '97', 1938.0, 7.9)
(196, 'Key Largo', 'John Huston', '', '', 100, '', 1948.0, 7.7)
(69, 'White Heat', 'Raoul Walsh', '', '', 114, '', 1949.0, 8.1)
(550, 'The War of the Worlds', 'Byron Haskin', '', '', 85, '78', 1953.0, 7.0)
(11, 'Shichinin no samurai', 'Akira Kurosawa', '', '', 207, '98', 1954.0, 8.6)
(273, 'Gojira', 'Ishirô Honda', '', '', 96, '78', 1954.0, 7.5)
(71, 'Kakushi-toride no san-akunin', 'Akira Kurosawa', '', '', 139, '89', 1958.0, 8.1)
(29, 'North by Northwest', 'Alfred Hitchcock', '', '', 136, '98', 1959.0, 8.3)
(179, 'The Magnificent Seven', 'John Sturg', '', '', 128, '74', 1960.0, 7.7)
(48, 'Yôjinbô', 'Akira Kurosawa', '', '', 110, '93', 1961.0, 8.2)
(272, 'The Gun

In [179]:
# 4) Display all movie names in descending order.
mycursor.execute("SELECT Movie_Name FROM movie_1 ORDER by Movie_Name DESC;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Ölümlü Dünya',)
('Æon Flux',)
('xXx: State of the Union',)
('xXx: Return of Xander Cage',)
('xXx',)
('Zoolander 2',)
('Zombieland: Double Tap',)
('Zombieland',)
('Zatôichi',)
('Zathura: A Space Adventure',)
("Zack Snyder's Justice League",)
('Yôjinbô',)
('Your Highness',)
('Young Guns II',)
('Young Guns',)
('You Only Live Twice',)
("You Don't Mess with the Zohan",)
('Yip Man chin chyun',)
('Yip Man 4',)
('Yip Man 3',)
('Yip Man 2',)
('Ying xiong',)
('Yi dai zong shi',)
('Yat goh ho yan',)
('X: First Class',)
('X2',)
('X-Men: The Last Stand',)
('X-Men: Days of Future Past',)
('X-Men: Apocalypse',)
('X-Men Origins: Wolverine',)
('X-Men',)
('Wyatt Earp',)
('Wrongfully Accused',)
('Wrath of the Titans',)
('Wrath of Man',)
('World War Z',)
('Wonder Woman 1984',)
('Wonder Woman',)
('WolfWalkers',)
('Wo hu cang long',)
('Without Remorse',)
('Windtalkers',)
("Willy's Wonderland",)
('Willow',)
('Wild Wild West',)
('Wild Target',)
('Wild Hogs',)
('Wild Card',)
('Whiteout',)
('White House Down'

In [180]:
# 5) Display movie name starts with ‘P’ and their rating is greater than 7
mycursor.execute("SELECT Movie_Name FROM movie_1 WHERE Movie_Name like 'P%' AND Ratings>7.0;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Paan Singh Tomar',)
('Pirates of the Caribbean: The Curse of the Black Pearl',)
('Per un pugno di dollari',)
('Predator',)
('Ponniyin Selvan: I',)
('Pushpa: The Rise - Part 1',)
("Pirates of the Caribbean: Dead Man's Chest",)
('Predestination',)
('Patriots Day',)
('Point Break',)
('Prey',)
("Pirates of the Caribbean: At World's End",)
('Payback',)
('Planet Terror',)
('Public Enemies',)
('Pitch Black',)
('Pacific Rim',)
('Pineapple Express',)
('Patriot Games',)
('Pandorum',)
("Pete's Dragon",)
('Pirates of the Caribbean: On Stranger Tides',)
('Prince of Persia: The Sands of Time',)
('Penguins of Madagascar',)
('Plane',)
('Pirates of the Caribbean: Dead Men Tell No Tales',)
('Pokémon: Detective Pikachu',)
('Peppermint',)
('Premium Rush',)
('Pain & Gain',)
('Predators',)
('Polar',)
('Predator 2',)
('Paycheck',)
('Proof of Life',)
('Pearl Harbor',)
('Parker',)
('Push',)
('Point of No Return',)
('Project Power',)
('Pathaan',)
('Power Rangers',)
('Passenger 57',)
('Punisher: War Zone',)
('

In [181]:
# 6) Display all movie names with star Arnold Schwarzenegger in ascending order.
sql_query = """
SELECT MoviesName 
FROM movie_2 
WHERE (Stars_1 in ('Arnold Schwarzenegge','Arnold Schwarzenegger') or Stars_2 in ('Arnold Schwarzenegge','Arnold Schwarzenegger'));
"""

mycursor.execute(sql_query)
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('249',)
('304',)
('695',)
('862',)
('673',)
('634',)
('1,329',)
('1,710',)
('2,000',)
('1,290',)
('1,469',)
('2,523',)
('1,643',)
('2,712',)
('4,830',)
('2,660',)
('2,917',)
('3,697',)
('4,635',)
('4,393',)
('',)
('2,959',)
('448',)


In [182]:
# 7) Display all details of the movie with the highest number of votes.
mycursor.execute("SELECT * FROM movie_2 WHERE Votes=(SELECT max(Votes) FROM movie_2);")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('3,923', 'The Last Witch Hunter', 'Vin Diesel', 'Rose Leslie', 'Elijah Wood', 'Ólafur Darri Ólafsson', '107,053', 'Action', 'Adventure', 'Fantasy', '27.37', 'A')


In [183]:
# 8) Display movie names with gross collections in descending order.
mycursor.execute("SELECT MoviesName,Gross FROM movie_2 WHERE Gross !='nan' ORDER by Gross DESC;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('2,573', 'Western')
('1,262', 'Western')
('4,861', 'Western')
('1,427', 'Western')
('1,899', 'Western')
('2,412', 'Western')
('3,000', 'Western')
('2,523', 'Western')
('305', 'War')
('2,298', 'War')
('', 'War')
('', 'War')
('306', 'War')
('2,813', 'War')
('593', 'War')
('', 'War')
('31', 'War')
('1,668', 'War')
('289', 'War')
('', 'War')
('', 'War')
('3,981', 'War')
('', 'War')
('3,313', 'War')
('2,740', 'War')
('3,747', 'War')
('4,783', 'War')
('3,982', 'War')
('111', 'Thriller')
('', 'Thriller')
('587', 'Thriller')
('', 'Thriller')
('', 'Thriller')
('3', 'Thriller')
('518', 'Thriller')
('2,084', 'Thriller')
('2,592', 'Thriller')
('', 'Thriller')
('', 'Thriller')
('1,050', 'Thriller')
('1,083', 'Thriller')
('4,225', 'Thriller')
('', 'Thriller')
('29', 'Thriller')
('829', 'Thriller')
('951', 'Thriller')
('1,765', 'Thriller')
('18', 'Thriller')
('936', 'Thriller')
('1,927', 'Thriller')
('2,392', 'Thriller')
('2,976', 'Thriller')
('4,540', 'Thriller')
('', 'Thriller')
('', 'Thriller')
(

In [184]:
# 9) Display the gross collection of movies with the star Arnold.
mycursor.execute("""SELECT MoviesName,Gross FROM movie_2
WHERE Stars_1 like '%Arnold%' or  Stars_2 like '%Arnold%' or  Stars_3 like '%Arnold%' or  Stars_4 like '%Arnold%';""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('249', '')
('304', '')
('695', 'Horror')
('862', 'Sci-Fi')
('673', 'Thriller')
('634', 'Fantasy')
('1,002', '')
('1,329', 'Thriller')
('1,710', 'Thriller')
('2,000', 'Comedy')
('1,290', '')
('1,469', 'Sci-Fi')
('2,523', 'Western')
('738', 'Sci-Fi')
('1,643', 'Crime')
('2,712', 'Thriller')
('4,830', 'Crime')
('2,660', 'Fantasy')
('2,917', 'Sci-Fi')
('3,697', 'Horror')
('4,635', 'Drama')
('4,393', 'Thriller')
('', 'Thriller')
('2,959', 'Fantasy')
('448', '')


In [185]:
# 10) Display all details of movies with comedy and action genres.
mycursor.execute("""SELECT * FROM movie_2
WHERE  Genre_1 in ('Action','Comedy') AND Genre_2 in ('Action','Comedy') or Genre_3 in ('Action','Comedy');""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('13', 'Spider-Man: Across the Spider-Verse', 'hameik Moore', 'Hailee Steinfeld', 'Brian Tyree Henry', 'Luna Lauren Velez', '195,199', 'Animation', 'Action', 'Adventure', '', 'U')
('89', 'Spider-Man: Into the Spider-Verse', 'hameik Moore', 'Jake Johnson', 'Hailee Steinfeld', 'Mahershala Ali', '613,892', 'Animation', 'Action', 'Adventure', '190.24', 'U')
('1,132', 'Mononoke-hime', 'Yôji Matsud', 'Yuriko Ishida', 'Yûko Tanaka', 'Billy Crudup', '411,402', 'Animation', 'Action', 'Adventure', '2.38', 'U')
('3,130', 'Kimetsu no Yaiba: Mugen Ressha-Hen', 'Natsuki Hanae', 'Akari Kitô', 'Yoshitsugu Matsuoka', 'Hiro Shimono', '66,421', 'Animation', 'Action', 'Adventure', '47.70', 'UA')
('', 'Gangs of Wasseypur', 'Manoj Bajpayee', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia', 'Richa Chadha', '100,011', 'Action', 'Comedy', 'Crime', '', 'A')
('', 'Sherlock Jr.', 'Buster Keaton', 'Kathryn McGuire', 'Joe Keaton', 'Erwin Connelly', '53,325', 'Action', 'Comedy', 'Romance', '0.98', '')
('1,223', 'Lock, Sto

In [191]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [203]:
import mysql.connector

In [204]:
# 11) Display all details from both tables where movie names are the same.
mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_1.Movie_Name in (select Movie_Name from (SELECT Movie_Name , count(*) as count from movie_1
group by Movie_Name
HAVING count>1)) ;""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

In [195]:
# 12) Display all movie names, Director, ratings, and gross collection where the genre is action.
mycursor.execute("""SELECT Movie_Name,Director_1,Director_2,Director_3,Ratings,movie_2.Gross FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE Genre_1='Action' or Genre_2='Action' or Genre_3='Action';""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('65', 'Scott Beck', 'Bryan Woods', '', 2023.0, 'Drama')


In [201]:
# 13) Display all details from both tables with the highest gross collection.
mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_2.Gross=(SELECT max(movie_2.Gross) FROM movie_2 WHERE movie_2.Gross!='nan');""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

In [197]:
# 14) Display all details from both tables with the highest ratings.
mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_1.Ratings=(SELECT max(movie_1.Ratings) FROM movie_1);""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

In [198]:
# 15) Display all details from both tables with the lowest gross collection and lowest ratings.
mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_2.Gross=(SELECT min(movie_2.Gross) FROM movie_2)
ORDER by movie_1.Ratings LIMIT 1;""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)